# Import Model

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
!pip install accelerate loralib bitsandbytes datasets git+https://github.com/huggingface/peft.git transformers sentencepiece

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-mo4d3w3y
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-mo4d3w3y
  Resolved https://github.com/huggingface/peft.git to commit f5352f08c593503a442f4438e1c7e648377b6a4b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.3 MB/s eta 0:00:

In [ ]:
import os
import sys
import torch
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig
from peft import PeftModel
from typing import List


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


In [ ]:
# LLaMATokenizer is the old class name, no unexpected tokenization will be produced using LlamaTokenizer

torch.autocast("cuda")

tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")
model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_4bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

# Generation Test


In [ ]:
def generate_prompt(instruction, input=None):
    if input:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        {instruction}

        ### Input:
        {input}
        ### Response:"""
    else:
        return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
        ### Instruction:
        {instruction}
        ### Response:"""




In [ ]:
generation_config = GenerationConfig(
    temperature=1.0,
    top_p=1,
    top_k=40,
    num_beams=4
)

In [ ]:
instruction = "From the description of a rule: identify the 'trigger', identify the 'action', write a IF 'trigger' THEN 'action' rule."

input = "When Foobot detects pollution, the system should turn on the appliance plugged into your WeMo Switch."

prompt = generate_prompt(instruction=instruction, input=input)
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
input_ids = inputs["input_ids"].cuda()

generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    output_scores=True,
    max_new_tokens=256
)

In [ ]:
output = tokenizer.batch_decode(generation_output, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0].split("### Response:")[1].strip()
print(output)
print("IF Foobot Pollution is high THEN WeMo Smart Plug Turn on")

When Foobot detects pollution, the system should turn on the appliance plugged into your WeMo Switch.

        ### Instruction:
        From the description of a rule: identify the 'trigger', identify the 'action', write a IF 'trigger' THEN 'action' rule.

        ### Input:
        When Foobot detects pollution, the system should turn on the appliance plugged into your WeMo Switch.
IF Foobot Pollution is high THEN WeMo Smart Plug Turn on


In [ ]:
trigger = output.split("THEN")[0].strip().replace('IF','').replace('(','').replace(')','').split('AND')
action = output.split("THEN")[1].strip().replace('(','').replace(')','').split('AND')
print(trigger)
print(action)

["When Foobot detects pollution, the system should turn on the appliance plugged into your WeMo Switch.\n\n        ### Instruction:\n        From the description of a rule: identify the 'trigger', identify the 'action', write a  'trigger'"]
["'action' rule.\n\n        ### Input:\n        When Foobot detects pollution, the system should turn on the appliance plugged into your WeMo Switch."]


# Prompter

In [ ]:
"""
A dedicated helper to manage templates and prompt building.
"""

import json
import os.path as osp
from typing import Union


class Prompter(object):
    __slots__ = ("template", "_verbose")

    def __init__(self, template_name: str = "", verbose: bool = False):
        self._verbose = verbose
        if not template_name:
            # Enforce the default here, so the constructor can be called with '' and will not break.
            template_name = "alpaca"
        file_name = "/content/alpacal.json"
        if not osp.exists(file_name):
            raise ValueError(f"Can't read {file_name}")
        with open(file_name) as fp:
            self.template = json.load(fp)
        if self._verbose:
            print(
                f"Using prompt template {template_name}: {self.template['description']}"
            )

    def generate_prompt(
        self,
        instruction: str,
        input: Union[None, str] = None,
        label: Union[None, str] = None,
    ) -> str:
        # returns the full prompt from instruction and optional input
        # if a label (=response, =output) is provided, it's also appended.
        if input:
            res = self.template["prompt_input"].format(
                instruction=instruction, input=input
            )
        else:
            res = self.template["prompt_no_input"].format(
                instruction=instruction
            )
        if label:
            res = f"{res}{label}"
        if self._verbose:
            print(res)
        return res

    def get_response(self, output: str) -> str:
        return output.split(self.template["response_split"])[1].strip()


# Fine Tuning

In [ ]:
tokenizer.pad_token_id = (
    0  # unk. we want this to be different from the eos token
)
tokenizer.padding_side = "left"  # Allow batched inference

In [ ]:
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = prompter.generate_prompt(
        data_point["instruction"],
        data_point["input"],
        data_point["output"],
    )
    tokenized_full_prompt = tokenize(full_prompt)
    if not train_on_inputs:
        user_prompt = prompter.generate_prompt(
            data_point["instruction"], data_point["input"]
        )
        tokenized_user_prompt = tokenize(
            user_prompt, add_eos_token=add_eos_token
        )
        user_prompt_len = len(tokenized_user_prompt["input_ids"])

        if add_eos_token:
            user_prompt_len -= 1

        tokenized_full_prompt["labels"] = [
            -100
        ] * user_prompt_len + tokenized_full_prompt["labels"][
            user_prompt_len:
        ]  # could be sped up, probably
    return tokenized_full_prompt


In [ ]:
# model/data params
base_model: str = "decapoda-research/llama-7b-hf"  # the only required argument
data_path: str =  'AntonioRenatoMontefusco/trainFor4090'
output_dir: str = "/content/drive/MyDrive/outputModel"
# training hyperparams
batch_size: int = 128
micro_batch_size: int = 8
num_epochs: int = 1
learning_rate: float = 3e-4
cutoff_len: int = 512
val_set_size: int = 500
# lora hyperparams
lora_r: int = 16
lora_alpha: int = 16
lora_dropout: float = 0.05
lora_target_modules: List[str] = ["q_proj","k_proj","v_proj","o_proj"]
# llm hyperparams
train_on_inputs: bool = True  # if False masks out inputs in loss
add_eos_token: bool = False
group_by_length: bool = False  # faster but produces an odd training loss curve
# wandb params
wandb_project: str = ""
wandb_run_name: str = ""
wandb_watch: str = ""  # options: false | gradients | all
wandb_log_model: str = ""  # options: false | true
resume_from_checkpoint: str = None  # either training checkpoint or final adapter
prompt_template_name: str = "alpaca"  # The prompt template to use will default to alpaca.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
gradient_accumulation_steps = batch_size // micro_batch_size

prompter = Prompter(prompt_template_name)

device_map = "auto"
world_size = int(os.environ.get("WORLD_SIZE", 1))
ddp = world_size != 1
if ddp:
    device_map = {"": int(os.environ.get("LOCAL_RANK") or 0)}
    gradient_accumulation_steps = gradient_accumulation_steps // world_size

# Check if parameter passed or if set within environ
use_wandb = len(wandb_project) > 0 or (
    "WANDB_PROJECT" in os.environ and len(os.environ["WANDB_PROJECT"]) > 0
)
# Only overwrite environ if wandb param passed
if len(wandb_project) > 0:
    os.environ["WANDB_PROJECT"] = wandb_project
if len(wandb_watch) > 0:
    os.environ["WANDB_WATCH"] = wandb_watch
if len(wandb_log_model) > 0:
    os.environ["WANDB_LOG_MODEL"] = wandb_log_model

In [ ]:
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)

In [ ]:
model = prepare_model_for_int8_training(model)

config = LoraConfig(
    r=lora_r,
    lora_alpha=lora_alpha,
    target_modules=lora_target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)

In [ ]:
from datasets import load_dataset

In [ ]:
data = load_dataset(data_path)
if val_set_size > 0:
    train_val = data["train"].train_test_split(
        test_size=val_set_size, shuffle=True, seed=42
    )
    train_data = (
        train_val["train"].shuffle().map(generate_and_tokenize_prompt)
    )
    val_data = (
        train_val["test"].shuffle().map(generate_and_tokenize_prompt)
    )
else:
    train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = None

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/AntonioRenatoMontefusco___json/AntonioRenatoMontefusco--trainFor4090-1f2286f1db4f1ea2/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/360124 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
trainer = transformers.Trainer(
          model=model,
          train_dataset=train_data,
          eval_dataset=val_data,
          args=transformers.TrainingArguments(
              per_device_train_batch_size=micro_batch_size,
              gradient_accumulation_steps=gradient_accumulation_steps,
              warmup_steps=100,
              num_train_epochs=num_epochs,
              learning_rate=learning_rate,
              fp16=True,
              logging_steps=10,
              optim="adamw_torch",
              evaluation_strategy="steps" if val_set_size > 0 else "no",
              save_strategy="steps",
              eval_steps=5 if val_set_size > 0 else None,
              save_steps=5,
              output_dir=output_dir,
              save_total_limit=3,
              load_best_model_at_end=True if val_set_size > 0 else False,
              ddp_find_unused_parameters=False if ddp else None,
              group_by_length=group_by_length,
              report_to="wandb" if use_wandb else None,
              run_name=wandb_run_name if use_wandb else None,
          ),
          data_collator=transformers.DataCollatorForSeq2Seq(
              tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
          ),
      )

In [ ]:
model.config.use_cache = False

old_state_dict = model.state_dict
model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))

if torch.__version__ >= "2" and sys.platform != "win32":
    model = torch.compile(model)

In [ ]:
trainer.train(resume_from_checkpoint=resume_from_checkpoint)

Step,Training Loss,Validation Loss
5,No log,2.560088


In [ ]:
model.save_pretrained(output_dir)

print(
    "\n If there's a warning about missing keys above, please disregard :)"
)